In [ ]:
import torch

# feature_map = torch.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple neural network structure
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=0)
        self.fc = nn.Linear(4, 1)  # Assuming the output of conv is flattened to 4 elements

    def forward(self, x):
        x = F.sigmoid(self.conv(x))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize the model
model = SimpleConvNet()

# Define inputs
input_feature_map = torch.randn(1, 1, 4, 4)  # Batch size of 1, 1 channel, 4x4 feature map
true_label = torch.tensor([[[17.]]])  # Placeholder for 'yy'

# Forward pass
output = model(input_feature_map)
loss_fn = nn.MSELoss()
loss = loss_fn(output, true_label)

# Backward pass
model.zero_grad()
loss.backward()

# Gradients with respect to the weight and input feature
gradient_wrt_w11 = model.conv.weight.grad[0, 0, 0, 0]
gradient_wrt_a00 = input_feature_map.grad[0, 0, 0, 0]

print(f"Gradient with respect to w11: {gradient_wrt_w11}")
print(f"Gradient with respect to input feature a00: {gradient_wrt_a00}")


/opt/miniconda3/envs/casper_env/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: 'NoneType' object is not subscriptable

In [4]:
from sympy import symbols, diff, exp, Function

# Define symbols
w11, a00, yy = symbols('w11 a00 yy')
x = symbols('x')
sigma = Function('sigma')(x)
y_pred = symbols('y_pred')  # Placeholder for the output of the network

# Define the sigmoid function and its derivative
sigma = 1 / (1 + exp(-x))
d_sigma_dx = diff(sigma, x)

# Assuming the output y_pred is a function of w11 and a00 through some operation, we need to define y_pred
# For simplicity, let's assume y_pred = sigma(w11 * a00), which is not the actual case but simplifies the gradient computation
# In reality, y_pred would depend on the entire convolution operation and all weights

# MSE Loss
mse = (y_pred - yy)**2 / 2
# Gradient of MSE loss w.r.t. y_pred
d_mse_dy_pred = diff(mse, y_pred)

# Let's simplify the gradients with the assumptions
# Assuming a direct relation for simplification: y_pred = sigma(w11 * a00)
y_pred_simple = sigma.subs(x, w11 * a00)

# Gradients of interest
d_loss_dw11 = diff(mse.subs(y_pred, y_pred_simple), w11)
d_loss_da00 = diff(mse.subs(y_pred, y_pred_simple), a00)

d_loss_dw11, d_loss_da00


(a00*(-yy + 1/(1 + exp(-a00*w11)))*exp(-a00*w11)/(1 + exp(-a00*w11))**2,
 w11*(-yy + 1/(1 + exp(-a00*w11)))*exp(-a00*w11)/(1 + exp(-a00*w11))**2)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

yy=10.

# Assume a00 and w11 are the parameters of interest
# Initialize input feature map and convolution kernel
input_feature = torch.randn(1, 1, 4, 4, requires_grad=True)  # 1 input channel, 4x4 size, requires_grad enabled for input
conv_kernel = torch.randn(1, 1, 3, 3, requires_grad=True)  # 1 output channel, 1 input channel, 3x3 kernel size
target = torch.tensor([yy])  # Assuming yy is a known scalar value

# Convolution operation
conv = F.conv2d(input_feature, conv_kernel, bias=None)
# Sigmoid activation
activated_output = torch.sigmoid(conv)
# Flatten
flattened_output = activated_output.view(-1)
# Assume a simple direct regression value from the flattened output for simplicity
# In practice, this would involve a fully connected layer with weights
regression_value = flattened_output.sum()

# MSE Loss
loss = F.mse_loss(regression_value.unsqueeze(0), target)  # Unsqueezing to match dimensions
# Backpropagate
loss.backward()

# Gradients
grad_w11 = conv_kernel.grad[0, 0, 1, 1]  # Gradient with respect to w11, the middle value of the kernel
grad_a00 = input_feature.grad[0, 0, 0, 0]  # Gradient with respect to a00, the top-left value of the input feature

grad_w11, grad_a00


(tensor(-1.0904), tensor(-0.0037))